# Softmax函数
将神经网络输出值转换到0-1之间的概率值

# MINST数据集分类任务实战

In [11]:
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from torchvision import datasets,transforms
from torch.utils.data import dataloader
import math

In [12]:
train_data=datasets.MNIST(root='.',train=True,transform=transforms.ToTensor(),download=True)
test_data=datasets.MNIST(root='.',train=False,transform=transforms.ToTensor(),download=True)

In [13]:
batch_size=64 # 每次训练传入数据量
# 装载,将全部数据通过一定方式打包成batches，每个batch里有固定数量的数据
train_loader=dataloader.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_loader=dataloader.DataLoader(dataset=test_data,batch_size=batch_size,shuffle=True)


In [14]:
for i,data in enumerate(train_loader):
    inputs,labels=data
    print(inputs.shape) #数据量，通道数，单个数据数组大小
    print(labels.shape) #标签个数
    break
print(len(train_loader))# 每次拿出一个装载好的batch进行处理（训练），共有938个batch

torch.Size([64, 1, 28, 28])
torch.Size([64])
938


In [15]:
# 定义网络
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(784,10)# 784输入，10输出
        self.softmax=nn.Softmax(dim=1) 
    def forward(self,x):
        # 需要转换数据为二维数据
        x=x.view(x.size()[0],-1)
        x=self.fc1(x)
        x=self.softmax(x)
        return x

In [16]:
model=Net()
lr=0.05
loss=nn.MSELoss()
opt=torch.optim.SGD(model.parameters(),lr=lr)

In [17]:
def train():
    for i,data in enumerate(train_loader):
        inputs,label=data
        out=model(inputs)
        # 转换标签输出为独热编码
        label=label.reshape(-1,1)
        one_hot=torch.zeros(inputs.shape[0],10).scatter(1,label,1)
        #scatter三个参数解释：dim变换维度,index：将哪个位置变成src值；src：插入的值
        Loss=loss(out,one_hot)# 两个参数大小要一致
        opt.zero_grad()
        Loss.backward()
        opt.step()
def test():
    correct=0
    for i,data in enumerate(test_loader):
        inputs,labels=data
        out=model(inputs)
        _,predicted=torch.max(out,1)# max返回最大值和最大值的位置
        # 计算召回率，准确率
        correct += (predicted==labels).sum()
    print("accuracy:{}".format(correct.item()/len(test_data)))

In [18]:
for epoch in range(10):
    print("epoch:",epoch)
    train()
    test()

epoch: 0
accuracy:0.675
epoch: 1
accuracy:0.8125
epoch: 2
accuracy:0.8512
epoch: 3
accuracy:0.8641
epoch: 4
accuracy:0.8724
epoch: 5
accuracy:0.8773
epoch: 6
accuracy:0.8813
epoch: 7
accuracy:0.8842
epoch: 8
accuracy:0.8875
epoch: 9
accuracy:0.8893
